## Testing TensorFlow off the shelf classifiers

Classification Modules:
-  [Insects Module](https://tfhub.dev/google/aiy/vision/classifier/insects_V1/1)
-  [Birds Module](https://tfhub.dev/google/aiy/vision/classifier/birds_V1/1)
-  [Plants Module 1](https://tfhub.dev/google/aiy/vision/classifier/plants_V1/1)

In [ ]:
# np==1.21.5
# tensorflow==2.8.0
# keras==2.8.0
# tensorflow_hub==0.12.0


In [1]:
import csv
from dataclasses import dataclass
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
from numpy.linalg import norm

from keras.preprocessing import image


#### Define Functions

In [ ]:
def normalize_01(input_image):
    """Normalise image to [0, 1]"""
    return input_image/255.

def load_img_to_array(img_path, target_size):
    img = image.load_img(img_path, target_size = target_size)
    img = image.img_to_array(img)
    return img

In [ ]:
def preprocess_image(img_path, target_size=(224, 224)):
    """Load image to arary, resize, scale and expand dimensions."""
    img = load_img_to_array(img_path, target_size)
    assert img.shape == (*target_size, 3)
    assert np.min(img) >= 0
    assert np.max(img) <= 255.

    # scale to [0, 1]
    img = normalize_01(img)
    assert np.min(img) >= 0
    assert np.max(img) <= 1.

    # expand dim
    img = np.expand_dims(img, axis = 0)
    assert img.shape == (1, *target_size, 3)

    return img

In [ ]:
def load_labelmap_to_dict(csv_path):
    labeldict = {}
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            labeldict[int(row['id'])] = row['name']
    return labeldict

In [ ]:
@dataclass
class ModelOutput:
    name: str 
    raw_score: float
    scaled_score: float 


def get_top_n_preds(model_output, labeldict, n=5, scale_threshold=1e-2):
    top_n_indices = (-model_output).argsort()[:n]

    model_outputs = []
    for ni in top_n_indices:
        sum_scale = sum(model_output[model_output>scale_threshold])
        model_outputs.append(ModelOutput(
            name = labeldict[ni],
            raw_score = (model_output[ni] / np.sum(model_output)),
            scaled_score = (model_output[ni] / sum_scale)
        ))
    return model_outputs

In [ ]:
def get_top_n_preds_dict(model_output, labeldict, n=5):
    top_n_indices = (-model_output).argsort()[:n]
    model_outputs = {labeldict[ni] : (model_output[ni] / np.sum(model_output)) for ni in top_n_indices}
    return model_outputs

#### Birds
Only 964 bird species in `bird_labelmap`

In [ ]:
# download keras model
m = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/birds_V1/1')

# load label dictionary
bird_labeldict = load_labelmap_to_dict('./bird_labelmap.csv')

# preprocess image and run through model
test_img = preprocess_image('./birdy.jpg')
test_img_embeds = m(test_img).numpy()[0]

# get outputs
outputs = get_top_n_preds(test_img_embeds, bird_labeldict, 5)
for o in outputs:
    print(o)

#### Insects
`1021` insect species to be identified

In [ ]:
# download keras model
m_insects = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/insects_V1/1')

# load label dictionary
insect_labeldict = load_labelmap_to_dict('./insects_labelmap.csv')

# preprocess image and run through model
test_img = preprocess_image('./ladybird.jpg')
test_img_embeds = m_insects(test_img).numpy()[0]

# get outputs
outputs = get_top_n_preds(test_img_embeds, insect_labeldict, 5)
for o in outputs:
    print(o)

#### Plants
`2101` species already

In [ ]:
# download keras model
m_plants = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/plants_V1/1')

# load label dictionary
plants_labeldict = load_labelmap_to_dict('./plants_labelmap.csv')

# preprocess image and run through model
test_img = preprocess_image('./snakey.jpeg')
test_img_embeds = m_plants(test_img).numpy()[0]

# get outputs
outputs = get_top_n_preds(test_img_embeds, plants_labeldict, 5)
for o in outputs:
    print(o)